In [ ]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# 그래프 기본 설정
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False
import matplotlib as mpl
import matplotlib.font_manager as fm

# 폰트 파일 경로 지정
font_path = '/content/drive/MyDrive/font/NanumGothic.ttf'

# 폰트 등록 및 matplotlib 설정
font_name = fm.FontProperties(fname=font_path).get_name()
fm.fontManager.addfont(font_path)
mpl.rc('font', family=font_name)
# 경고 뜨지 않게
import warnings
warnings.filterwarnings('ignore')

#저장라이브러리
import pickle

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 표준화
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# LabelEncoder
from sklearn.preprocessing import LabelEncoder

# 데이터를 학습용과 검증용으로 나눈다.
from sklearn.model_selection import train_test_split

import os

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/데이터톤/data/주력_비주력상품 데이터/top3_cleaned.parquet')
df

,event_type,product_id,category_id,brand,price,user_id,user_session,event_date,event_hour,category_lvl1,category_lvl2,category_lvl3,category_lvl4,weekday
0,view,3900821,2053013552326770905,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,2019-10-01,5,appliances,environment,water_heater,None,Tuesday
1,view,1004237,2053013555631882655,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,2019-10-01,5,electronics,smartphone,None,None,Tuesday
2,view,1004545,2053013555631882655,huawei,566.01,537918940,406c46ed-90a4-4787-a43b-59a410c1a5fb,2019-10-01,5,electronics,smartphone,None,None,Tuesday
3,view,2900536,2053013554776244595,elenberg,51.46,555158050,b5bdd0b3-4ca2-4c55-939e-9ce44bb50abd,2019-10-01,5,appliances,kitchen,microwave,None,Tuesday
4,view,1005011,2053013555631882655,samsung,900.64,530282093,50a293fb-5940-41b2-baf3-17af0e812101,2019-10-01,5,electronics,smartphone,None,None,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188464725,view,1004225,2232732093077520756,apple,1104.28,512572600,3a243e5a-d4a3-4cf1-afae-a671c7f37cd4,2020-03-31,23,construction,tools,light,None,Tuesday
188464726,view,5701087,2053013554415534427,jvc,56.63,561157123,1d4320ed-aa16-4196-8129-b59bb7bce726,2020-03-31,23,electronics,video,tv,None,Tuesday
188464727,view,100132075,2053013554658804075,lenovo,566.27,616307331,774c99a4-8fdd-44a3-ae2e-e6a1b4f6a2ee,2020-03-31,23,electronics,audio,headphone,None,Tuesday
188464728,view,100010158,2053013563835941749,pulser,363.41,625382291,bcd82fde-b570-45e9-83f6-a8fc1579a40f,2020-03-31,23,appliances,kitchen,refrigerators,None,Tuesday


# 1회 구매자 데이터 저장(주력상품)

In [ ]:
# 구매 이벤트만 필터링하여 user별 구매 횟수 계산
purchase_counts = df[df['event_type'] == 'purchase'].groupby('user_id').size()

# 한 번만 구매한 고객 리스트 추출
one_time_buyers = purchase_counts[purchase_counts == 1].index

Index([101875240, 136662675, 177507559, 210849689, 217628234, 221480173,
       229414998, 229576190, 238445570, 241307693,
       ...
       635086978, 635088886, 635089105, 635090869, 635091137, 635091320,
       635091571, 635093230, 635094924, 635096039],
      dtype='int64', name='user_id', length=690159)

In [ ]:
# 전체 이벤트에서 구매 1회 고객만 필터링
one_time_df = df[df['user_id'].isin(one_time_buyers)]
one_time_df

,event_type,product_id,category_id,brand,price,user_id,user_session,event_date,event_hour,category_lvl1,category_lvl2,category_lvl3,category_lvl4,weekday
18,view,3701244,2053013565983425517,elenberg,33.44,515535834,d90a6f2f-0d0f-47cf-9fe9-62db93dffdb9,2019-10-01,5,appliances,environment,vacuum,None,Tuesday
20,view,3600575,2053013563810775923,hotpoint-ariston,275.37,554754045,bd0302ef-c5ca-4b6a-b916-95cc2840c72c,2019-10-01,5,appliances,kitchen,washer,None,Tuesday
36,view,1004870,2053013555631882655,samsung,286.86,516489361,7d6d03fb-39d6-4143-8694-981a550acaaa,2019-10-01,5,electronics,smartphone,None,None,Tuesday
38,view,27700113,2053013560086233771,leo,48.90,515630204,f9cc0313-5572-4894-a4eb-45d855f064cc,2019-10-01,5,construction,tools,pump,None,Tuesday
53,view,3900774,2053013552326770905,artel,72.85,513664804,c0177eae-2279-4b2f-8bf7-e9837cc0612a,2019-10-01,5,appliances,environment,water_heater,None,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188464697,purchase,6800658,2053013555220840837,kingston,53.77,624595867,f71468a4-7ca7-453b-954f-f5e72cba556a,2020-03-31,23,appliances,kitchen,juicer,None,Tuesday
188464699,view,1005098,2232732093077520756,samsung,145.18,634646276,41e61625-2646-4b06-b9b0-9b4cc13c9959,2020-03-31,23,construction,tools,light,None,Tuesday
188464707,view,5800861,2232732082390433922,kicx,36.55,628330088,ad624635-dd1b-480a-867a-2fda5c0a23a2,2020-03-31,23,electronics,audio,subwoofer,None,Tuesday
188464708,view,100039953,2232732099754852875,kivi,257.12,549607680,d92aa32a-2a69-4d97-8259-352bd181806a,2020-03-31,23,appliances,personal,massager,None,Tuesday


In [ ]:
one_time_df['event_type'].unique()

array(['view', 'purchase', 'cart'], dtype=object)

In [ ]:
# Parquet 형식 저장 (더 빠르고 용량 효율적)
one_time_df.to_parquet('/content/drive/MyDrive/Colab Notebooks/데이터톤/data/주력_비주력상품 데이터/one_time_buyers_all_events.parquet', index=False)

# 또는 CSV로 저장하고 싶다면
# one_time_df.to_csv('one_time_buyers_all_events.csv', index=False)


# 두번 이상 구매한 고객의 데이터(주력상품)

In [ ]:
# 구매 이벤트만 필터링해서 user_id별 구매 횟수 계산
purchase_counts = df[df['event_type'] == 'purchase'].groupby('user_id').size()

# 2번 이상 구매한 고객 ID 추출
multi_buyers = purchase_counts[purchase_counts >= 2].index

In [ ]:
multi_buyer_df = df[df['user_id'].isin(multi_buyers)]

In [ ]:
# Parquet 형식으로 저장
multi_buyer_df.to_parquet('/content/drive/MyDrive/Colab Notebooks/데이터톤/data/주력_비주력상품 데이터/multi_buyers_all_events.parquet', index=False)


### 보기만하고 구매하지 않은 고객의 수 확인

In [ ]:
# 구매한 고객 목록
buyers = df[df['event_type'] == 'purchase']['user_id'].unique()

# 보기(view)한 고객 목록
viewers = df[df['event_type'] == 'view']['user_id'].unique()

# 보기만 하고 구매 안 한 고객
view_only_users = set(viewers) - set(buyers)

# 고객 수
num_view_only_users = len(view_only_users)
print(f"보기만 하고 구매하지 않은 고객 수: {num_view_only_users}")

보기만 하고 구매하지 않은 고객 수: 8474991


In [ ]:
df2 = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/데이터톤/data/주력_비주력상품 데이터/other_cleaned.parquet')
df2

,event_type,product_id,category_id,brand,price,user_id,user_session,event_date,event_hour,category_lvl1,category_lvl2,category_lvl3,category_lvl4,weekday
0,view,17200506,2053013559792632471,None,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8,2019-10-01,5,furniture,living_room,sofa,None,Tuesday
1,view,1307067,2053013558920217191,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,2019-10-01,5,computers,notebook,None,None,Tuesday
2,view,1480613,2053013561092866779,pulser,908.62,512742880,0d0d91c2-c9c2-4e81-90a5-86594dec0db9,2019-10-01,5,computers,desktop,None,None,Tuesday
3,view,28719074,2053013565480109009,baden,102.71,520571932,ac1cd4e5-a3ce-4224-a2d7-ff660a105880,2019-10-01,5,apparel,shoes,keds,None,Tuesday
4,view,13500240,2053013557099889147,brw,93.18,555446365,7f0062d8-ead0-4e0a-96f6-43a0b79a2fc4,2019-10-01,5,furniture,bedroom,bed,None,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98131528,view,100172862,2232732114124538702,rooman,71.82,626088076,77472822-64b5-4000-a1e3-b757b4244466,2020-03-31,23,apparel,shoes,None,None,Tuesday
98131529,view,7202279,2232732084546306225,None,9.52,634136524,f0d03600-507a-4f26-b4e0-67aa1fa2c6c2,2020-03-31,23,furniture,living_room,chair,None,Tuesday
98131530,view,9200473,2232732104343421549,ritmix,7.00,630903806,267c010f-8dbd-4cd4-8816-f4ffea502d73,2020-03-31,23,apparel,scarf,None,None,Tuesday
98131531,view,15500139,2232732108319621820,flight,73.36,517441190,ac227671-933a-43f8-b057-f70df41f2dd7,2020-03-31,23,apparel,underwear,None,None,Tuesday


# 1회 구매자 데이터 저장(비주력상품)

In [ ]:
# 구매 이벤트만 필터링하여 user별 구매 횟수 계산
purchase_counts_2 = df2[df2['event_type'] == 'purchase'].groupby('user_id').size()

# 한 번만 구매한 고객 리스트 추출
one_time_buyers_2 = purchase_counts_2[purchase_counts_2 == 1].index

In [ ]:
# 전체 이벤트에서 구매 1회 고객만 필터링
one_time_df_2 = df2[df2['user_id'].isin(one_time_buyers_2)]
one_time_df_2


,event_type,product_id,category_id,brand,price,user_id,user_session,event_date,event_hour,category_lvl1,category_lvl2,category_lvl3,category_lvl4,weekday
34,view,28714755,2053013565228450757,respect,51.22,552469659,02b2fcfa-2e63-4b51-81bb-4995e7cd1767,2019-10-01,5,apparel,shoes,None,None,Tuesday
55,view,6000229,2053013560807654091,starline,138.23,518090724,3c63ed7c-0739-4121-9363-370c3a453a34,2019-10-01,5,auto,accessories,alarm,None,Tuesday
81,view,6000229,2053013560807654091,starline,138.23,518090724,3c63ed7c-0739-4121-9363-370c3a453a34,2019-10-01,5,auto,accessories,alarm,None,Tuesday
85,view,27000002,2053013564674802599,housefit,460.76,523185540,19016b1d-72b4-43af-9cd2-216061797861,2019-10-01,5,sport,trainer,None,None,Tuesday
100,view,4700645,2053013560899928785,fujida,162.17,516331621,b85c1a5c-42da-4d34-8822-e276eb70a3b9,2019-10-01,5,auto,accessories,videoregister,None,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98131498,view,16600201,2232732108780995272,None,308.89,614107343,92874bed-87db-4ad4-b597-9f887b4e9627,2020-03-31,23,sport,trainer,None,None,Tuesday
98131500,view,1201471,2232732101407408685,samsung,463.05,568459058,93739c13-1737-4597-8461-3eddfe2f14c2,2020-03-31,23,apparel,shoes,slipons,None,Tuesday
98131503,view,27100015,2232732112304210726,None,8.49,561315651,5142a12d-8994-4d26-837e-6ba8a9672bfe,2020-03-31,23,kids,swing,None,None,Tuesday
98131521,cart,7600170,2232732103982711397,d-link,7.70,592982145,e719af53-7b0c-44e6-b347-f31fb3df06e8,2020-03-31,23,furniture,kitchen,table,None,Tuesday


In [ ]:
# Parquet 형식 저장 (더 빠르고 용량 효율적)
one_time_df_2.to_parquet('/content/drive/MyDrive/Colab Notebooks/데이터톤/data/주력_비주력상품 데이터/비주력상품/noTop3_one_time_buyers_all_events.parquet', index=False)

# 또는 CSV로 저장하고 싶다면
# one_time_df.to_csv('one_time_buyers_all_events.csv', index=False)


# 두번 이상 구매한 고객의 데이터(비주력상품)

In [ ]:
# 구매 이벤트만 필터링해서 user_id별 구매 횟수 계산
purchase_counts_22 = df2[df2['event_type'] == 'purchase'].groupby('user_id').size()

# 2번 이상 구매한 고객 ID 추출
multi_buyers_22 = purchase_counts_22[purchase_counts_22 >= 2].index

In [ ]:
multi_buyer_df_22 = df2[df2['user_id'].isin(multi_buyers_22)]

In [ ]:
# Parquet 형식으로 저장
multi_buyer_df_22.to_parquet('/content/drive/MyDrive/Colab Notebooks/데이터톤/data/주력_비주력상품 데이터/noTop3_multi_buyers_all_events.parquet', index=False)

# ✅ 주력/비주력 상품 데이터 확인

In [ ]:
# df_1 = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/데이터톤/data/주력_비주력상품 데이터/주력상품/one_time_buyers_all_events.parquet')
df_2 = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/데이터톤/data/주력_비주력상품 데이터/비주력상품/noTop3_one_time_buyers_all_events.parquet의 사본')
# df_3 = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/데이터톤/data/주력_비주력상품 데이터/주력상품/multi_buyers_all_events.parquet')
df_4 = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/데이터톤/data/주력_비주력상품 데이터/비주력상품/noTop3_multi_buyers_all_events.parquet의 사본')

In [ ]:
df_2['category_lvl1'].unique()

array(['apparel', 'auto', 'sport', 'computers', 'furniture',
       'accessories', 'kids', 'medicine', 'country_yard', 'stationery'],
      dtype=object)

In [ ]:
df_4['category_lvl1'].unique()

array(['computers', 'furniture', 'auto', 'apparel', 'stationery', 'sport',
       'kids', 'accessories', 'country_yard', 'medicine'], dtype=object)